In [ ]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# % matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.utils import shuffle
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
import operator


Using TensorFlow backend.


In [2]:
excel_data = pd.read_excel('data/oczyszczone.xls', sheet_name = 'daneSOB')
headers = excel_data.columns.values
excel_data.head()
print(headers)

['Płeć' 'Wiek' 'Lokalizacja bólu na początku zachorowania'
 'Lokalizacja bólu obecnie' 'Intensywność bólu' 'Czynniki nasilające ból'
 'Czynniki przynoszące ulgę' 'Progresja bólu' 'Czas trwania bólu'
 'Charakter bólu na początku zachorowania' 'Charakter bólu obecnie'
 'Nudności i wymioty' 'Apetyt' 'Wypróżnienia' 'Oddawanie moczu'
 'Poprzednie niestrawności' 'Żółtaczka w przeszłości'
 'Poprzednie operacje brzuszne' 'Leki' 'Stan psychiczny' 'Skóra'
 'Temperatura (pacha)' 'Tętno' 'Ruchy oddechowe powłok brzusznych'
 'Wzdęcia' 'Umiejscowienie bolesności uciskowej' 'Objaw Blumberga'
 'Obrona mięśniowa' 'Wzmożone napięcie powłok brzusznych'
 'Opory patologiczne' "Objaw Murphy'ego" 'Klasy diagnozy']


In [3]:
excel_data.groupby('Klasy diagnozy').size()

Klasy diagnozy
1    141
2     17
3     29
4     28
5     55
6     32
7    157
8     17
dtype: int64

In [4]:
Y_data = excel_data['Klasy diagnozy']
X_data = excel_data.drop(columns = 'Klasy diagnozy')

pearsons = {}
for column in X_data.columns:
    pearsons[column] = pearsonr(X_data[column],Y_data)
# pearsons = pearsons.sort()
sorted_pearsons = sorted(pearsons.items(), key=operator.itemgetter(1), reverse=True)
sorted_keys = [x for x, y in sorted_pearsons]
# print(sorted_pearsons)
print(sorted_keys)

['Leki', 'Intensywność bólu', 'Czas trwania bólu', 'Stan psychiczny', 'Tętno', "Objaw Murphy'ego", 'Umiejscowienie bolesności uciskowej', 'Wypróżnienia', 'Lokalizacja bólu obecnie', 'Wzmożone napięcie powłok brzusznych', 'Poprzednie operacje brzuszne', 'Charakter bólu na początku zachorowania', 'Lokalizacja bólu na początku zachorowania', 'Wiek', 'Płeć', 'Temperatura (pacha)', 'Opory patologiczne', 'Żółtaczka w przeszłości', 'Poprzednie niestrawności', 'Apetyt', 'Wzdęcia', 'Progresja bólu', 'Obrona mięśniowa', 'Oddawanie moczu', 'Czynniki nasilające ból', 'Skóra', 'Charakter bólu obecnie', 'Ruchy oddechowe powłok brzusznych', 'Czynniki przynoszące ulgę', 'Objaw Blumberga', 'Nudności i wymioty']


In [5]:
# # headers = headers.tolist()
# for x in sorted_keys[30:]:
#     excel_data = excel_data.drop(columns = x)
# headers = excel_data.columns.values

In [6]:
diagnosis_classes = len(excel_data.groupby('Klasy diagnozy').size())
diagnosis_classes = [ str(x + 1) for x in range(diagnosis_classes)]
diagnosis_classes

['1', '2', '3', '4', '5', '6', '7', '8']

In [7]:
excel_data = shuffle(excel_data)
# one hot encoding
for i in headers:
    le = LabelEncoder()
    le.fit(excel_data[str(i)])
    excel_data[str(i)] = le.transform(excel_data[str(i)])
    encoded = to_categorical(excel_data[str(i)])
    
    if len(encoded[0]) > 2 and i != 'Klasy diagnozy':
        insert_loc = np.where(headers==i)
        insert_loc = insert_loc[0][0]

        excel_data = excel_data.drop(columns = i)
        for label in range(len(encoded[0])):
            excel_data.insert(int(insert_loc), str(i) + "_" + str(label), encoded[:,label])
    

excel_data.head()


,Płeć,Wiek_4,Lokalizacja bólu na początku zachorowania_10,Lokalizacja bólu obecnie_10,Intensywność bólu_2,Czynniki nasilające ból_3,Czynniki przynoszące ulgę_2,Progresja bólu_2,Czas trwania bólu_3,Charakter bólu na początku zachorowania_2,...,Poprzednie operacje brzuszne,Leki,Ruchy oddechowe powłok brzusznych,Wzdęcia,Objaw Blumberga,Obrona mięśniowa,Wzmożone napięcie powłok brzusznych,Opory patologiczne,Objaw Murphy'ego,Klasy diagnozy
145,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,1,0,1,0,0,0,0,1
308,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0,0,0,0,1,0,0,1,0,6
43,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,1,1,1,0,0
178,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0,0,1,1,1,0,1,0,0,2
52,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,1,1,1,0,1,0,0,0


In [8]:
# X_data set with arguments
# Y_data set with diagnostick class

Y_data = excel_data['Klasy diagnozy']
X_data = excel_data.drop(columns = 'Klasy diagnozy')


In [9]:
X_data.head()

,Płeć,Wiek_4,Lokalizacja bólu na początku zachorowania_10,Lokalizacja bólu obecnie_10,Intensywność bólu_2,Czynniki nasilające ból_3,Czynniki przynoszące ulgę_2,Progresja bólu_2,Czas trwania bólu_3,Charakter bólu na początku zachorowania_2,...,Żółtaczka w przeszłości,Poprzednie operacje brzuszne,Leki,Ruchy oddechowe powłok brzusznych,Wzdęcia,Objaw Blumberga,Obrona mięśniowa,Wzmożone napięcie powłok brzusznych,Opory patologiczne,Objaw Murphy'ego
145,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,1,0,1,0,0,0,0
308,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,1,0
43,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,1,1,1,0
178,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0,0,0,1,1,1,0,1,0,0
52,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,1,1,1,0,1,0,0


In [10]:
# Zastanowić się nad dzieleniem zbioru wg klas diagnozy
# def shuffle_by_diagnostic_class(excel_data):
#    pass
  
# shuffle_by_diagnostic_class(excel_data)    

In [11]:
# ratio 1/2 to 1/2
X_train = X_data[:238]
Y_train = Y_data[:238]

X_test = X_data[238:]
Y_test = Y_data[238:]

kfold = [[X_train, Y_train, X_test, Y_test], [ X_test, Y_test, X_train, Y_train]]
cvscores = []

number_of_neurons = int(input("Podaj liczbę neuronów: "))
momentum_value = float(input("Podaj wartość momentum, wartość zero oznacza brak momentu, dopuszczalne liczby zmiennoprzecinkowe: "))


model = Sequential()
model.add(Dense(units = 106,activation = 'relu',input_dim = 106)) # warstwa wejściowa
model.add(Dense(units = number_of_neurons, activation = 'relu')) # warstwa ukryta
model.add(Dense(units = 8, activation = 'softmax')) # warstwa wyjściowa

Podaj liczbę neuronów: 8
Podaj wartość momentum, wartość zero oznacza brak momentu, dopuszczalne liczby zmiennoprzecinkowe: 0


In [12]:
for X_train, Y_train, X_test, Y_test in kfold:
   

    model.compile(optimizer= SGD(lr = 0.07, momentum=momentum_value),loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])
    model.fit(X_train, Y_train, epochs = 30,shuffle = True, verbose = 2, validation_data=(X_test, Y_test))
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Train on 238 samples, validate on 238 samples
Epoch 1/30
 - 0s - loss: 1.8976 - acc: 0.3403 - val_loss: 1.7241 - val_acc: 0.3025
Epoch 2/30
 - 0s - loss: 1.6352 - acc: 0.4034 - val_loss: 1.6638 - val_acc: 0.2983
Epoch 3/30
 - 0s - loss: 1.5205 - acc: 0.4034 - val_loss: 1.5775 - val_acc: 0.3361
Epoch 4/30
 - 0s - loss: 1.3987 - acc: 0.5126 - val_loss: 1.4692 - val_acc: 0.4454
Epoch 5/30
 - 0s - loss: 1.2396 - acc: 0.6050 - val_loss: 1.3175 - val_acc: 0.5756
Epoch 6/30
 - 0s - loss: 1.0814 - acc: 0.6723 - val_loss: 1.1952 - val_acc: 0.5798
Epoch 7/30
 - 0s - loss: 0.9439 - acc: 0.6891 - val_loss: 1.0618 - val_acc: 0.6261
Epoch 8/30
 - 0s - loss: 0.8356 - acc: 0.7143 - val_loss: 0.9767 - val_acc: 0.6429
Epoch 9/30
 - 0s - loss: 0.7427 - acc: 0.7311 - val_loss: 0.8980 - val_acc: 0.6891
Epoch 10/30
 - 0s - loss: 0.6715 - acc: 0.7773 - val_loss: 0.8426 - val_acc: 0.7269
Epoch 11/30
 - 0s - loss: 0.6092 - acc: 0.8151 - val_loss: 0.8244 - val_acc: 0.7395
Epoch 12/30
 - 0s - loss: 0.5535 - acc:

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 106)               11342     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 856       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
Total params: 12,270
Trainable params: 12,270
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.04)
y_pred = classifier.fit(X_train, Y_train).predict(X_test)
cnf_matrix = confusion_matrix(Y_test, y_pred)

In [15]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=diagnosis_classes,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=diagnosis_classes, normalize=True,
                      title='Normalized confusion matrix')

plt.show()


Confusion matrix, without normalization
[[60  0  0  1  0  0  1  0]
 [ 9  0  0  0  0  0  1  0]
 [ 0  0 13  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0]
 [ 0  0  0  0 24  0  0  0]
 [ 0  0  0  0  0 18  2  0]
 [ 0  0  0  0  0  0 88  0]
 [ 0  0  0  0  0  0  0  9]]
Normalized confusion matrix
[[0.97 0.   0.   0.02 0.   0.   0.02 0.  ]
 [0.9  0.   0.   0.   0.   0.   0.1  0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.9  0.1  0.  ]
 [0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.  ]]
